In [17]:
import numpy as np
import torch
from torch.utils import data

def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))


In [18]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [19]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [20]:
next(iter(data_iter))

[tensor([[-1.2712, -0.5317],
         [ 0.6237, -0.2406],
         [-0.6524,  1.2162],
         [ 0.1607,  0.6426],
         [ 0.3261,  0.3826],
         [-0.6030,  1.2546],
         [-0.2439, -1.2003],
         [-0.5729,  0.0331],
         [ 0.1121, -2.6809],
         [ 1.0511, -1.4717]]),
 tensor([[ 3.4482],
         [ 6.2697],
         [-1.2338],
         [ 2.3379],
         [ 3.5690],
         [-1.2529],
         [ 7.7958],
         [ 2.9377],
         [13.5290],
         [11.2911]])]

In [21]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

In [22]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [23]:
loss = nn.MSELoss()

In [24]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [25]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000231
epoch 2, loss 0.000100
epoch 3, loss 0.000100


In [30]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0001,  0.0006])
b的估计误差： tensor([0.0006])
